In [1]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize,RegexpTokenizer
import string
import nltk
from nltk.corpus import stopwords
#import en_core_web_sm
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import make_scorer, accuracy_score,recall_score,roc_auc_score,f1_score,classification_report

In [2]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

Read the reviews

In [3]:
rev=pd.read_csv('yelp_review_exp.csv',encoding='UTF-8')

/home/vishwa/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Filter the category of Thai Restaurant reviews

In [4]:
rev1=rev[rev.cat=='Restaurants Thai']

Mark the target to predict,If the review is useful or not

In [5]:
rev1['target']=np.where(rev1['useful']>0, 1, 0)

/home/vishwa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
rev1=rev1.reset_index()

Tokenize and remove stop words

In [7]:
rev1['tokenized_text'] = rev1['text'].apply(lambda x:x.lower()).apply(word_tokenize) 
punctuation = list(string.punctuation)
stopWords = set(stopwords.words('english'))
docs=list(rev1['tokenized_text'])


In [8]:
filtered_docs=[]
for i in docs:
    filt = []
    for w in i:
        if w.lower()not in stopWords and w not in punctuation:
            filt.append(w)
    filtered_docs.append(filt)

#nlp = en_core_web_sm.load()

final_txt=filtered_docs

Generate wor2vec model

In [9]:
 model = gensim.models.Word2Vec(final_txt, min_count=1, \
                                       workers=18, \
                                       window=10, \
                                       size=150, \
                                       hs=1, \
                                       negative=5, \
                                       sg=0, \
                                       iter=10)

In [10]:
model.wv.save_word2vec_format('embedding.txt')

In [11]:
def word2idx(word):
    return model.wv.vocab[word].index

def idx2word(idx):
    return model.wv.index2word[idx]

In [12]:
max_sentence_len=100

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


/home/vishwa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Conver reviews to Sequence

In [15]:
token_obj=Tokenizer()
token_obj.fit_on_texts(final_txt)
sequence=token_obj.texts_to_sequences(final_txt)

In [16]:
word_index=token_obj.word_index

Perform padding to reduce the review length to 100

In [17]:
rev_pad=pad_sequences(sequence,maxlen=max_sentence_len)

Create test train split

In [19]:
validation_split=.3
indices=np.arange(rev_pad.shape[0])
np.random.shuffle(indices)
label=rev1['target'][indices]

In [20]:
num_val_samp=int(validation_split*rev_pad.shape[0])
num_val_samp

10903

In [21]:
X_train=rev_pad[:-num_val_samp]
X_test=rev_pad[-num_val_samp:]
y_train=label[:-num_val_samp]
y_test=label[-num_val_samp:]

In [22]:
vocab_size=len(corpus)+1

Create embedding_matrix from the word2vec model


embedding_matrix = np.zeros((vocab_size, 150))
for word, i in token_obj.word_index.items():
    embedding_vector = model[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,GRU,Embedding


Create the keras metric function

In [25]:
def as_keras_metric(method):
    import functools
    from tensorflow.python.keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

In [26]:
#from tensorflow.keras import backend
#from tensorflow.python.keras import backend as k
import tensorflow as tf
auc_roc = as_keras_metric(tf.metrics.auc)
recall = as_keras_metric(tf.metrics.recall)

Running the GRU Neural Network model

In [27]:
smodel=Sequential()

In [28]:
emb_layer = Embedding(vocab_size, 150, weights=[embedding_matrix], input_length=max_sentence_len, trainable=False)

In [29]:
smodel.add(emb_layer)

In [30]:
smodel.add(GRU(units=64,dropout=.2,recurrent_dropout=.2))

In [31]:
smodel.add(Dense(units=1, activation='sigmoid'))
smodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc_roc])
smodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 150)          6804900   
_________________________________________________________________
gru (GRU)                    (None, 64)                41280     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 6,846,245
Trainable params: 41,345
Non-trainable params: 6,804,900
_________________________________________________________________


In [32]:
smodel.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.05, verbose=1)

Train on 24170 samples, validate on 1273 samples
Epoch 1/10
24170/24170 [==============================] - 167s 7ms/step - loss: 0.6981 - auc: 0.4864 - val_loss: 0.6847 - val_auc: 0.4970
Epoch 2/10
24170/24170 [==============================] - 156s 6ms/step - loss: 0.6822 - auc: 0.5065 - val_loss: 0.6842 - val_auc: 0.5108
Epoch 3/10
24170/24170 [==============================] - 162s 7ms/step - loss: 0.6745 - auc: 0.5184 - val_loss: 0.6869 - val_auc: 0.5244
Epoch 4/10
24170/24170 [==============================] - 168s 7ms/step - loss: 0.6705 - auc: 0.5312 - val_loss: 0.6880 - val_auc: 0.5357
Epoch 5/10
24170/24170 [==============================] - 153s 6ms/step - loss: 0.6667 - auc: 0.5412 - val_loss: 0.6862 - val_auc: 0.5453
Epoch 6/10
24170/24170 [==============================] - 152s 6ms/step - loss: 0.6612 - auc: 0.5510 - val_loss: 0.6923 - val_auc: 0.5547
Epoch 7/10
24170/24170 [==============================] - 149s 6ms/step - loss: 0.6563 - auc: 0.5601 - val_loss: 0.6942 - v

In [33]:
y_pred=smodel.predict_classes(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score
print('Accuracy of model on test data: ',roc_auc_score(y_test,y_pred))
print('Confusion Matrix: \n',confusion_matrix(y_test,y_pred))

Accuracy of model on test data:  0.502295369252242
Confusion Matrix: 
 [[4635 1794]
 [3205 1269]]
